In [3]:
# coding: utf-8

import glob
import os
import pandas as pd
import numpy as np
import shutil
from pathlib import Path

sddir = "/home/maik/b2drop/cosmicsense/inbox/marquardt/timeseries/crns/sd"
remotedir = "/home/maik/b2drop/cosmicsense/inbox/marquardt/timeseries/crns/remote"
trgdir = "/media/x/cosmicsense/data/marquardt/crns"
tmpfile = "tmpfile.txt"
tmpfile2 = "tmpfile2.txt"
tmpfile3 = "tmpfile3.txt"
ids = [1, 2, 4, 21, 22, 26, 27, 28]

crns = {
     1: {"remotepattern": "up1_Data*.001*.txt",
         "sdpattern": "*.001",
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt", "counts1", "nsecs1", "counts2", "nsecs2"],
         "colnames2": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt", "counts1", "nsecs1","counts2", "nsecs2", "sdiAdr","sdi1_1","sdi1_2","sdi1_3","sdi1_4"]
        },

     2: {"remotepattern": "up2_Data*.002*.txt",
         "sdpattern": "*.002",
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt", "counts1", "nsecs1"],
         "colnames2": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt", "counts1", "nsecs1","sdiAdr","sdi1_1","sdi1_2","sdi1_3","sdi1_4","sdi1_5","sdi1_6"]
        },

     4: {"remotepattern": "up4_Data*.004*.txt",
         "sdpattern": "*.004",
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt",
                      "counts1", "nsecs1", "counts2", "nsecs2", "MetOne092_1","press4","temp_ext",
                      "relhum_ext","N1T_C","N1RH","N2T_C","N2RH"],
         "colnames2": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt",
                      "counts1", "nsecs1", "counts2", "nsecs2", "MetOne092_1","press4","temp_ext",
                      "relhum_ext","N1T_C","N1RH","N2T_C","N2RH","sdiAdr","sdi2_1","sdi2_2","sdi2_3","sdi2_4"]
        },

    21: {"remotepattern": "CRSProbe_Data*.021*.txt",
          "sdpattern": "*.021",
          "colnames": ["rec_id", "datetime", "press1", "press4", "temp1","relhum1", "temp_ext",
                       "relhum_ext", "volt", "counts1", "nsecs1", "N1T_C", "N1RH"],
          "colnames2": ["rec_id", "datetime", "press1", "press4", "temp1","relhum1", "temp_ext",
                       "relhum_ext", "volt", "counts1", "nsecs1", "N1T_C", "N1RH","sdiAdr","sdi1_1","sdi1_2","sdi1_3","sdi1_4","sdi1_5","sdi1_6"]
        },

    22: {"remotepattern": "CRSProbe_Data*.022*.txt",
         "sdpattern": "*.022",
         "colnames": ["rec_id", "datetime", "press1", "press4", "temp1","relhum1", "temp_ext",
                       "relhum_ext", "volt", "counts1", "nsecs1", "N1T_C", "N1RH"],
         "colnames2": ["rec_id", "datetime", "press1", "press4", "temp1","relhum1", "temp_ext",
                       "relhum_ext", "volt", "counts1", "nsecs1", "N1T_C", "N1RH","sdiAdr","sdi1_1","sdi1_2","sdi1_3","sdi1_4","sdi1_5","sdi1_6"],
         "colnames3": ["rec_id", "datetime", "press1", "press4", "temp1","relhum1", "temp_ext",
                       "relhum_ext", "volt", "countsloc1", "nsecsloc1", "counts1", "nsecs1", "N1T_C", "N1RH","sdiAdrloc1","sdiloc1_1","sdiloc1_2","sdiloc1_3","sdiloc1_4","sdiAdr","sdi1_1","sdi1_2","sdi1_3","sdi1_4","sdi1_5","sdi1_6"]
        },

    26: {"remotepattern": "up26_Data*.026*.txt",
         "sdpattern": "*.026",
         "colnames": ["rec_id", "datetime", "press1", "press4", "temp1", "relhum1", "volt", "counts1", "nsecs1", "counts2", "nsecs2"]
        },

    27: {"remotepattern": "up27_Data*.027*.txt",
         "sdpattern": "*.027",
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1",
                      "volt", "counts1", "nsecs1", "counts2", "nsecs2"],
         "colnames2": ["rec_id", "datetime", "press1", "temp1", "relhum1",
                      "volt", "counts1", "nsecs1", "counts3", "nsecs3", "counts2", "nsecs2"]

        },

    28: {"remotepattern": "sonde28_Data_*.028*",
         "sdpattern": "*.028",
         "colnames": ["rec_id", "datetime", "press1", "press4", "temp1", "relhum1", "volt", "counts1", "nsecs1", "temp2", "relhum2"],
         "colnames2": ["rec_id", "datetime", "press1", "press4", "temp1", "relhum1", "volt", "counts1", "nsecs1", "temp2", "relhum2","sdiAdr","sdi1_1","sdi1_2","sdi1_3","sdi1_4"]
        }
}

In [ ]:
# Find all column descriptors FROM FILE
for i, id in enumerate(ids):
    print("-------------")
    print("Processing %d" % id)

    # REMOTE FILES
    print("   Remote: ", end="")
    searchdir = os.path.join(remotedir,"%d" % id, crns[id]["remotepattern"])
    remotefiles = glob.glob(searchdir, recursive=True)
    print("found %d files" % len(remotefiles), end="")
    
    allcols = []
    numwithcols = 0

    for name in remotefiles:
        foundcol = False
        #print(".", end="")
        with open(name, 'r') as fin:
            numiterations = 0
            while(numiterations<10000):
                line = fin.readline()
                # EOF
                if not line:
                    fin.close()
                    break
                if "RecordNum" in line:
                    # This is the colummn descriptor line
                    foundcol = True
                    line = line.strip(",")
                    line = line.replace(",\r\n", "")
                    line = line.replace(",\n", "")
                    line = line.strip("//")
                    fcols = line.split(",")
                    numcols = len(fcols)
                    for fcol in fcols:
                        fcol = fcol.strip("\n").strip("//")
                        if not fcol=="":
                            allcols.append(fcol)
                    fin.close()
                    break
            numwithcols += foundcol
                
    crns[id]["allcols"] = np.unique(allcols)
    crns[id]["numfiles"] = len(remotefiles)
    crns[id]["numnocols"] = len(remotefiles) - numwithcols
    print(", no column specs for %d files" % (len(remotefiles) - numwithcols) )

In [1]:
import sys

In [4]:
crns = {
     1: {"remotepattern": "up1_Data*.001*.txt",
        },

     2: {"remotepattern": "up2_Data*.002*.txt",
        },

     4: {"remotepattern": "up4_Data*.004*.txt",
        },

    21: {"remotepattern": "CRSProbe_Data*.021*.txt",
        },

    22: {"remotepattern": "CRSProbe_Data*.022*.txt",
        },

    26: {"remotepattern": "up26_Data*.026*.txt",
        },

    27: {"remotepattern": "up27_Data*.027*.txt",
        },

    28: {"remotepattern": "sonde28_Data_*.028*",
        }
}

# Find all column changes FROM FILE
for i, id in enumerate(ids):
    print("-------------")
    print("Processing %d" % id)

    # REMOTE FILES
    print("   Remote: ", end="")
    searchdir = os.path.join(remotedir,"%d" % id, crns[id]["remotepattern"])
    remotefiles = sorted(glob.glob(searchdir, recursive=True))
    print("found %d files" % len(remotefiles), end="")
    
    numcols = 0
    newnumcols = []
    newcolsdate = []
    newcolsfile = []

    for name in remotefiles:
        foundcol = False
        # Clean file content
        with open(name, 'r') as fin:
            body = fin.read()
            # replace comment character
            body = body.replace("//", "#")
            # replace zombie line endings
            body = body.replace(",\r\n", "\r\n")
            # comment out these lines
            body = body.replace("CRS#1:", "#CRS#1")
            body = body.replace("CRS#2:", "#CRS#2")
            fin.close()
        # and write it to tmpfile
        with open(tmpfile, 'w') as fout:
            fout.write(body)
        # Read again as Pandas dataframe
        try:
            df = pd.read_csv(tmpfile, sep=",", comment="#", header=None, error_bad_lines=False, 
                             warn_bad_lines=True)
        except pd.errors.EmptyDataError:
            continue
        if len(df.columns) < 7:
            continue            
        try:
            df[1] = pd.to_datetime(df[1], format="%Y/%m/%d %H:%M:%S")
        except:
            dtmask = np.repeat(True,len(df))
            for k in range(len(df)):
                try:
                    _ = pd.to_datetime(df.loc[k,1], format="%Y/%m/%d %H:%M:%S")
                except ValueError: 
                    dtmask[k] = False
            df2 = df.loc[dtmask]
            if len(df2)==0:
                sys.exit()
            df = df2
            df[1] = pd.to_datetime(df[1], format="%Y/%m/%d %H:%M:%S")
        if len(df.columns) != numcols:
                newnumcols.append(len(df.columns))
                newcolsdate.append(df[1][0])
                newcolsfile.append(name)
                numcols=len(df.columns)
    crns[id]["setup"] = pd.DataFrame({"datetime":newcolsdate, "file": newcolsfile, "numcols": newnumcols})
    print("")

-------------
Processing 1
   Remote: found 1930 files
-------------
Processing 2
   Remote: found 1484 files
-------------
Processing 4
   Remote: found 2543 files
-------------
Processing 21
   Remote: found 2651 files

b'Skipping line 255: expected 20 fields, saw 30\nSkipping line 272: expected 20 fields, saw 29\nSkipping line 280: expected 20 fields, saw 21\nSkipping line 315: expected 20 fields, saw 30\nSkipping line 323: expected 20 fields, saw 29\nSkipping line 331: expected 20 fields, saw 29\nSkipping line 348: expected 20 fields, saw 34\nSkipping line 356: expected 20 fields, saw 33\nSkipping line 364: expected 20 fields, saw 31\nSkipping line 381: expected 20 fields, saw 24\nSkipping line 389: expected 20 fields, saw 30\nSkipping line 406: expected 20 fields, saw 31\nSkipping line 423: expected 20 fields, saw 26\nSkipping line 431: expected 20 fields, saw 27\nSkipping line 439: expected 20 fields, saw 37\nSkipping line 447: expected 20 fields, saw 31\nSkipping line 455: expected 20 fields, saw 27\nSkipping line 463: expected 20 fields, saw 22\nSkipping line 472: expected 20 fields, saw 28\nSkipping line 480: expected 20 fields, saw 22\nSkipping line 489: expected 20 fields, saw 22\nSkipping li


-------------
Processing 22
   Remote: found 2470 files

b'Skipping line 8: expected 28 fields, saw 29\nSkipping line 9: expected 28 fields, saw 29\nSkipping line 10: expected 28 fields, saw 29\nSkipping line 11: expected 28 fields, saw 29\nSkipping line 12: expected 28 fields, saw 29\nSkipping line 13: expected 28 fields, saw 29\nSkipping line 14: expected 28 fields, saw 29\nSkipping line 15: expected 28 fields, saw 29\nSkipping line 17: expected 28 fields, saw 29\nSkipping line 18: expected 28 fields, saw 29\nSkipping line 19: expected 28 fields, saw 29\nSkipping line 21: expected 28 fields, saw 29\nSkipping line 22: expected 28 fields, saw 29\nSkipping line 23: expected 28 fields, saw 29\n'
b'Skipping line 10: expected 23 fields, saw 27\nSkipping line 11: expected 23 fields, saw 27\nSkipping line 12: expected 23 fields, saw 27\nSkipping line 13: expected 23 fields, saw 27\nSkipping line 14: expected 23 fields, saw 27\nSkipping line 15: expected 23 fields, saw 27\nSkipping line 16: expected 23 fields, saw 27\nSkipping line 17: expected 23 


-------------
Processing 26
   Remote: found 2082 files
-------------
Processing 27
   Remote: found 1770 files
-------------
Processing 28
   Remote: found 814 files


In [5]:
for key in crns.keys():
    print("======================================================")
    print(key)
    print(crns[key]["setup"])
    print("")

1
             datetime                                               file  \
0 2019-07-25 10:39:58  /home/maik/b2drop/cosmicsense/inbox/marquardt/...   
1 2020-04-15 10:23:28  /home/maik/b2drop/cosmicsense/inbox/marquardt/...   
2 2021-01-19 09:28:36  /home/maik/b2drop/cosmicsense/inbox/marquardt/...   
3 2021-01-25 12:02:02  /home/maik/b2drop/cosmicsense/inbox/marquardt/...   

   numcols  
0       10  
1       15  
2       14  
3       15  

2
             datetime                                               file  \
0 2019-07-31 12:37:38  /home/maik/b2drop/cosmicsense/inbox/marquardt/...   
1 2020-04-07 11:01:39  /home/maik/b2drop/cosmicsense/inbox/marquardt/...   
2 2021-04-19 10:39:18  /home/maik/b2drop/cosmicsense/inbox/marquardt/...   
3 2021-06-04 11:11:34  /home/maik/b2drop/cosmicsense/inbox/marquardt/...   

   numcols  
0        8  
1       15  
2       10  
3       15  

4
             datetime                                               file  \
0 2019-08-06 13:25:33  /

In [33]:
name

'/home/maik/b2drop/cosmicsense/inbox/marquardt/timeseries/crns/remote/21/CRSProbe_Data2104010004.021_027577.txt'

In [23]:
try:
    pd.to_datetime("34234", format="%Y/%m/%d %H:%M:%S")
except TypeError:
    print("ii")
    

ValueError: time data 34234 doesn't match format specified

In [9]:
with open(name, 'r') as file:
    file.close()

In [7]:
for i, id in enumerate(ids):
    print("-------------")
    print("Processing %d" % id)

    try:
        os.remove(tmpfile)
        os.remove(tmpfile2)
    except:
        pass

    # REMOTE FILES
    print("Remote: ", end="")
    searchdir = os.path.join(remotedir,"%d" % id, crns[id]["remotepattern"])
    remotefiles = glob.glob(searchdir, recursive=True)
    print("found %d files" % len(remotefiles))

    for name in remotefiles:
        print(".", end="")
        fin = open(name, "r")
        body = fin.read()
        # replace comment character
        body = body.replace("//", "#")
        # replace zombie line endings
        body = body.replace(",\r\n", "\r\n")
        # comment out these lines
        body = body.replace("CRS#1:", "#CRS#1")
        body = body.replace("CRS#2:", "#CRS#2")
        myfile = open(tmpfile, 'a')
        myfile.write(body)
        myfile.close()
    print("")

    # SD
    print("SD: ", end="")
    searchdir = os.path.join(sddir, "%d" % id)
    sdfiles = [filename for filename in Path(searchdir).glob("**/"+crns[id]["sdpattern"])]
    print("found %d files" % len(sdfiles))

    for name in sdfiles:
        print(".", end="")
        fin = open(name, "r")
        body = fin.read()
        # replace comment character
        body = body.replace("//", "#")
        # replace zombie line endings
        body = body.replace(",\r\n", "\r\n")
        body = body.replace(",\n", "\n")
        # comment out these lines
        body = body.replace("CRS#1:", "#CRS#1")
        body = body.replace("CRS#2:", "#CRS#2")
        myfile = open(tmpfile, 'a')
        myfile.write(body)
        myfile.close()
    print("")

    if ("colnames2" in crns[id].keys()) or ("colnames3" in crns[id].keys()):
        # Read all lines. potentially varying no of columns
        myfile = open(tmpfile, 'r')
        lines = myfile.readlines()
        myfile.close()
        # Write in seperate files
        myfile = open(tmpfile, 'w')
        myfile2 = open(tmpfile2, 'w')
        myfile3 = open(tmpfile3, 'w')
        for line in lines:
            split = line.split(",")
            if len(split)==len(crns[id]["colnames"]):
                myfile.write(line+"\n")
            if len(split)==len(crns[id]["colnames2"]):
                myfile2.write(line+"\n")
            try:
                if len(split)==len(crns[id]["colnames3"]):
                    myfile3.write(line+"\n")
            except:
                pass
        myfile.close()
        myfile2.close()
        try:
            myfile3.close()
        except:
            pass

    # MERGE
    df = pd.read_csv(tmpfile, sep=",", comment="#", header=None, error_bad_lines=False, warn_bad_lines=True)
    df.columns = crns[id]["colnames"]
    if "colnames2" in crns[id].keys():
        try:
            df2 = pd.read_csv(tmpfile2, sep=",", comment="#", header=None,
                             error_bad_lines=False, warn_bad_lines=True)
            df2.columns = crns[id]["colnames2"]
            df = df2.append(df, sort=False)
        except:
            print("Problem in reading or appending data with diffferent column scenario")
            raise
    if "colnames3" in crns[id].keys():
        try:
            df3 = pd.read_csv(tmpfile3, sep=",", comment="#", header=None,
                             error_bad_lines=False, warn_bad_lines=True)
            df3.columns = crns[id]["colnames3"]
            df = df3.append(df, sort=False)
        except:
            print("Problem in reading or appending data with diffferent column scenario")
            raise
    df.datetime = pd.to_datetime(df.datetime, format="%Y/%m/%d %H:%M:%S")
    df = df.set_index("datetime")
    df.insert(loc=1, column="datetime", value=df.index)
    df = df.sort_index()
    df = df[df.index >= "2019-07-25"]
    dupl = df.index.duplicated(keep='first')
    if np.any(dupl):
        print("Contains %d duplicates" % len(np.where(dupl)[0]))
        df = df[~dupl]
    fpath = os.path.join(trgdir, "%d/%d_CRNS.txt" % (id, id) )
    df.to_csv(fpath, sep="\t", index=False, date_format="%Y-%m-%d %H:%M:%S")
    print("")

-------------
Processing 27
Remote: found 329 files
.........................................................................................................................................................................................................................................................................................................................................
SD: found 768 files
......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................